# Covid-19 Dashboard

This simple dashboard is based on Public Health England (PHE) data, which indicates the distribution of COVID-19 cases for males and females based on their age range. 

In [1]:
# This section's code is used by all the following sections
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

from uk_covid19 import Cov19API

In [2]:
# It includes graphs
%matplotlib inline
# Make figures larger
plt.rcParams['figure.dpi'] = 100

In [3]:
#It loads the JSON file and stores the raw data
data = {}
with open("agedistribution.json", "rt") as INFILE:
    data=json.load(INFILE)

In [4]:
# It converts the JSON file to data frame
def wrangle_data(rawdata):
    
    datadic=rawdata['data'][0]
    
    males=datadic['males']
    females=datadic['females']
    
    ageranges = [x['age'] for x in males]
    
    def min_age(agerange):
        agerange=agerange.replace('+','')
        start=agerange.split('_')[0]
        return int(start)
    
    ageranges.sort(key=min_age)
    age_df=pd.DataFrame(index=ageranges, columns=['males', 'females', 'total'])
    
    for entry in males:
        ageband=entry['age']
        age_df.loc[ageband,'males']=entry['value']
        
    for entry in females:
        ageband=entry['age']
        age_df.loc[ageband, 'females']=entry['value']
        
    age_df['total']=age_df['males']+age_df['females']
    
    return age_df

age_df=wrangle_data(data)

In [5]:
# This is accessesing the PHE API and returns raw data in the same format as data loaded from the "canned" JSON file.
def access_api():
    filters = [
    'areaType=nation',
    'areaName=England'
    ]
    
    structure = {
    "males": "maleCases",
    "females": "femaleCases"
    }
    api = Cov19API(filters=filters, structure=structure)
    agedistribution=api.get_json()
    return agedistribution


# The button callback accesses API, wrangles data, updates global variable df used for plotting.
def api_button_callback(button):
        apidata=access_api()
        global age_df
        age_df=wrangle_data(apidata)
        refresh_graph()
        apibutton.icon="check"
        
# Create a button to allow users to refresh the data
apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='info',
    tooltip="'Click to download current Public Health England data'",
    icon='refresh'
)


## Distribution of Covid-19 cases by gender and age graph

The following graph displays daily COVID-19 cases in England for males and females by dividing the data based on their age. In order to evaluate the latest data, please click on the 'Refresh Data' button. Therefore, an interactive control was added to select any combination of the males, females or total categories (CTRL-Click to select more than one category).

In [6]:
# Create a widget for the user to select columns
agecols=wdg.SelectMultiple(
    options=['males', 'females', 'total'],
    value=['males'],
    rows=3,
    description='Gender',
    disabled=False
)

# Create a widget for the user to select columns
def refresh_graph():
    current=agecols.value[0]
    if current==agecols.options[0]:
        other=agecols.options[1]
    else:
        other=agecols.options[0]
    agecols.value=(other,) 
    agecols.value=(current,) 
    
        
def age_graph(graphcolumns):
    ncols=len(graphcolumns)
    if ncols>0:
        age_df.plot(kind='bar', y=list(graphcolumns))
        plt.title("Distribution of Covid-19 cases by gender and age")
        plt.xlabel("Age Group")
        plt.ylabel("Total Number")
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
        


output=wdg.interactive_output(age_graph, {'graphcolumns': agecols})

apibutton.on_click(api_button_callback)
display(apibutton)
display(agecols, output)


Button(button_style='info', description='Refresh data', icon='refresh', style=ButtonStyle(), tooltip="'Click t…

SelectMultiple(description='Gender', index=(0,), options=('males', 'females', 'total'), rows=3, value=('males'…

Output()

**Author and Copyright Notice** Maria-Georgiana Duta (27/11/2020), *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*